In [ ]:
!pip install kaggle

In [ ]:
import kaggle

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [46]:
api = KaggleApi()
api.authenticate()

In [53]:
!kaggle competitions download titanic -f train.csv --force
!kaggle competitions download titanic -f test.csv --force


  0%|          | 0.00/59.8k [00:00<?, ?B/s]
100%|##########| 59.8k/59.8k [00:00<00:00, 1.76MB/s]



  0%|          | 0.00/28.0k [00:00<?, ?B/s]
100%|##########| 28.0k/28.0k [00:00<00:00, 1.34MB/s]


In [36]:
import pandas as pd

In [28]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_id = test["PassengerId"]

In [15]:
def clean(data):
    data=data.drop(["Ticket","Cabin","Name","PassengerId"], axis=1)
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace = True)
    data.Embarked.fillna("F", inplace=True)
    return data

In [16]:
data=clean(data)
test=clean(test)

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cols = ["Sex","Embarked"]
for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.fit_transform(test[col])

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [19]:
y=data["Survived"]
X=data.drop("Survived", axis=1)

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=42)
clf = LogisticRegression(random_state=0,max_iter=10000).fit(X_train,y_train)
predictions = clf.predict(X_val)

In [54]:
submission_preds = clf.predict(test)
submissions = pd.DataFrame({"PassengerId":test_id.values, "Survived": submission_preds,})
submissions.to_csv("submission.csv", index=False)

In [55]:
!kaggle competitions submit titanic -f submission.csv -m "submission script test"

Successfully submitted to Titanic - Machine Learning from Disaster



  0%|          | 0.00/3.18k [00:00<?, ?B/s]
100%|##########| 3.18k/3.18k [00:00<00:00, 29.8kB/s]
100%|##########| 3.18k/3.18k [00:01<00:00, 1.67kB/s]
